<a href="https://colab.research.google.com/github/Konstvv/iBio-Summer-School/blob/main/iBio_Summer_school_PPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Protein-Protein Interaction prediction using Keras

by Konstantin Volzhenin

27/07/2022

# Contents
- Introduction
- Dataset
- Protein representations and embeddings
- PPI prediction model
- Data Analysis
- C1 - C2 - C3 classes


# Introduction

## 1. PPI Networks

**Protein–protein interactions (PPIs)** are physical contacts established between two or more protein molecules as a result of biochemical events steered by various physical interactions. When studying a given organism (or an ecosystem with multiple organisms) the PPIs are usually combined together to form a protein-protein interaction network. Such networks or interactomes are of a great interest because of their value in different areas of protein related research (e.g. search for potential protein targets in therapeutic interest).


---

<p align="center">
  <img src="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4898894/bin/npjschz201612-f2.jpg" />
</p>

<center>
Fig. 1 Schizophrenia interactome
</center>
    
    Image taken from: Ganapathiraju MK, Thahir M, Handen A, Sarkar SN, Sweet RA, Nimgaonkar VL, Loscher CE, Bauer EM, Chaparala S (April 2016). "Schizophrenia interactome with 504 novel protein-protein interactions". NPJ Schizophrenia. 2: 16012.

---


## 2. Protein-protein interactions task - binary classification

During this class we will create a Deep Learning architechture for PPI prediction. The task itself is a binary classification: we have two sequences as an input and the model would have to estimate the probability of these two proteins to interact (a number between 0 and 1).

# Dataset
During this class we will work with PPI data
of yeast *Saccharomyces cerevisiae*. An
independent data set of 11 474 yeast PPIs contains approximately equal number of interacting and non-interacting pairs.

    Reference: Guo,Y. et al. (2008) Using support vector machine combined with auto covariance to predict protein–protein interactions from protein sequences. Nucleic Acids Res., 36, 3025–3030.

The dataset is composed of two files:

> **1. Interactions**

  Each entry contains two protein IDs and a binary label (1 - two given proteins interact, 0 - do not interact)


> **2. Dictionary**

  Contains protein IDs and corresponding protein sequences.
<br>
<br>

Before working with a model we have to upload and preprocess the data.



Let's open the two downloaded files as DataFrames using Pandas and see what they look like:

In [ ]:
!git clone https://github.com/Konstvv/iBio-Summer-School
import pandas as pd
import numpy as np

data = pd.read_csv('iBio-Summer-School/Interactions.tsv', sep='\t')
dict_seq = pd.read_csv('iBio-Summer-School/Dictionary.tsv', sep='\t')

print(data)
print(dict_seq)

# Protein representations and embeddings

The neural network cannot process the protein sequence represented by a string. In order to overcome this issue we have to find a way how to represent any given sequence numerically. In this class we will consider a couple of potential options



First of all let's evaluate the length distribution of our sequences in the dataset. 
1. Is this dataset balanced? What are the classes proportions?
2. Using *plt.hist* plot the sequence length distribution from *dict_seq*

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Enter your code here for the question 1

# Enter your code here for the question 2
# plt.hist()
# plt.show()

In [ ]:
## Introducing a couple of parameters

# The sequence size for our future model
# All the sequences will be either trimmed or extended to this size
sequence_size = 2000  #@param {type: "slider", min: 100, max: 4000}

# Vocabulary of aminoacids
aminoacids = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
                'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']

## 1. One-hot embedding

The features are encoded using a ‘one-of-K’ or ‘dummy’ scheme. Every letter is replaced by a vector of size *len(aminoacids)* which is filled with 0 with only one value (at position *aminoacids.index(letter)*) being 1.

Please write a function **one_hot_emb** that takes a protein sequence as a string input and returns a one-hot array with the size *(sequence_size, len(aminoacids))*. If the string is longer than *sequence_size*, delete the excess (shorter - pad the rest with zeros).

In [ ]:
def one_hot_emb(string):
    # A shape that has to be returned for every one-hot embedding: (sequence_size, len(aminoacids))
    seq_tensor = np.zeros((sequence_size, len(aminoacids)), dtype=np.float16)
    for i in range(len(string)):
        char = string[i]
        if char in aminoacids:
            seq_tensor[i, aminoacids.index(char)] = 1
        if i == sequence_size - 1:
            break
    return seq_tensor

## 2. NLF embedding

This method takes many physicochemical properties and transforms them using PCA (dimensionality reduction method) and Fisher transformation creating a small set of features that can describe an amino acid. There are 18 transformed features in total.

    Reference: L. Nanni and A. Lumini, “A new encoding technique for peptide classification,” Expert Syst. Appl., vol. 38, no. 4, pp. 3185–3191, 2011.

We can upload a DataFrame that contains the transrormed vectors for all the corresponding amino acids:

In [ ]:
nlf = pd.read_csv('https://raw.githubusercontent.com/dmnfarrell/epitopepredict/master/epitopepredict'
                        '/mhcdata/NLF.csv', index_col=0)
# The letter X will represent an empty position (added in case we might encounter empty spots somewhere)
nlf['X'] = [0.0] * nlf.shape[0]
# If we encounter Selenocysteine, we'll treat it as Cysteine
nlf['U'] = nlf['C']
print(nlf)

Here please write a function **nlf_emb** which would be similar to *one_hot_emb* and would take a protein sequence as an input and return an array with the size *(sequence_size, 18)* where each letter will be replaced by a vector taken from the *nlf* DataFrame

In [ ]:
#def nlf_emb(string):

    # A shape that has to be returned for every one-hot embedding: (sequence_size, nlf.shape[0])
    
    ## Uncomment the first line and enter your code here

## 3. Blosum embedding

BLOSUM62 is a substitution matrix that specifies the similarity of one amino acid to another by means of a score. This score reflects the frequency of substiutions found from studying protein sequence conservation in large databases of related proteins. The number 62 stands for the percentage identity at which sequences are clustered in the analysis.

We can take columns of this matrix which correspond to a particular amino acid as embedding vectors. These way puts the amino acids with similar physicochemical properties closer together in the embedding space.

In [ ]:
blosum = pd.read_csv('https://raw.githubusercontent.com/dmnfarrell/epitopepredict/master/epitopepredict'
                        '/mhcdata/blosum62.csv', index_col=0)
# If we encounter Selenocysteine, we'll treat it as Cysteine
blosum['U'] = blosum['C']
# We will not encounter B, Z or * so we can delete the corresponding vectors
blosum.drop(labels=['B', 'Z', 'X', '*'], axis=0, inplace=True)
blosum.drop(labels=['B', 'Z', '*'], axis=1, inplace=True)
print(blosum)

Here please write a function **blosum_emb** which would be similar to *one_hot_emb* and would take a protein sequence as an input and return an array with the size *(sequence_size, blosum.shape[0])*

In [ ]:
#def blosum_emb(string):

    # A shape that has to be returned for every one-hot embedding: (sequence_size, blosum.shape[0])
    
    ## Uncomment the first line and enter your code here

## 4. Other approaches

There are many other different ways of how one can create a sequence embedding. The ones that may provide a good performance can be heavier and more complicated than the examples we have today. For example, the good solutions are:


*   *Protein profiles*

Profiles are used to model protein families and domains. They are built by converting multiple sequence alignments into position-specific scoring systems (PSSMs). Amino acids at each position in the alignment are scored according to the frequency with which they occur.

*   Pretrained language models

Various neural networks can be used to compute the embeddings while being trained on a large number of proteins. Such algorithms may use different types of information: the structural and physicochemical similarity between proteins, pairwise residue contact maps for individual proteins, etc. Usually these methods can provide more generalized view than trainable embeddings inside the predictive model due to the fact that they have been trained on large amounts of data.  




Once we have written our embedding function we can use it to convert all the sequences in our protein dictionary into numpy arrays.
Let's replace the strings in the 'Seq_string' column of the *dict_seq* DataFrame with the corresponding embeddings using one of the functions above:

In [ ]:
## Choose the function for the embedding
emb = 'Blosum' #@param ['Blosum', 'NLF', 'One-Hot']

embs = {'Blosum' : blosum_emb, 'NLF': nlf_emb, "One-Hot": one_hot_emb}

dict_seq['Seq_string'] = dict_seq['Seq_string'].apply(embs[emb])
print(dict_seq)

Then, let's replace all the protein IDs in the *data* (columns 'Seq1' and 'Seq2') with corresponding matrices from the *dict_seq*:

In [ ]:
data = data.merge(dict_seq, left_on='Seq1', right_on='Id', how='left')
data = data.merge(dict_seq, left_on='Seq2', right_on='Id', how='left', suffixes=('1', '2'))

data = data[['Seq_string1', 'Seq_string2', 'Label']]

print(data)

# PPI prediction model
In this section we will create a binary classification model for PPI predition. We will do that using the keras module:

In [ ]:
from keras import layers
from keras import Model
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import gc

gpus = tf.config.list_physical_devices('GPU')

# Making sure we use GPU and setting the memory gorwth
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

## 1. Input - Embedding

Here we need to transform the data into numpy arrays and split it into 3 parts: training, validation, and testing. We will use training dataset for the training itself. The validation data will be used to evaluate the performance of the model after each epoch and to choose the best configuration. It is also might be used to choose the optimal values of hyperparameters. The testing data will be used afterwards to make a conclusion about how well the model performs.

In [ ]:
from sklearn.model_selection import train_test_split

x1 = np.asarray(data['Seq_string1'].tolist(), dtype=np.float16)
x2 = np.asarray(data['Seq_string2'].tolist(), dtype=np.float16)
y = np.asarray(data['Label'].tolist(), dtype=np.float32)



## Enter your code here to split the data into train and test sets using train_test_split:



# Delete the variables we do not need anymore to free some RAM in case the notebook crashes
del data, dict_seq, x1, x2, y

In [ ]:
input_dimensions = (sequence_size, x1_train.shape[-1])

# Inputs
seq_input1 = layers.Input(shape=input_dimensions, name='seq1')
seq_input2 = layers.Input(shape=input_dimensions, name='seq2')

## 2. Siamese module

The first part of the Neural network is a so-called siamese module which process two sequences separately. It takes a keras Tensor as an input and outputs another Tensor after having done several manupulations. In this class the Siamese module will consist of three types of layers: Convolutional, MaxPool, and GRU. 

Here below you can find a schematic gif of 1D convolution. It shows how exactly a filter is applied to the sequence to compute the output:

<p align="center">
  <img src="https://e2eml.school/images/conv1d/stride_2.gif" />
</p>

In the simplest case the output value of conv1D layer with input size $(N, L, C_{in})$ and output size $(N, L_{out}, C_{out})$ can be calculated using the following formula:

$$
out(N_i, C_{out_j}) = bias(C_{out_j}) + ∑_{k=0}^{C_{in}-1}weight(C_{out_j}, k) ⋆ input(N_i, k)
$$

where $⋆$ is the cross-correlation operator.

Write the function that processes a Tensor by using 3-4 Conv+MaxPool layers followed by a Bidirectional GRU. Make sure to initialize the layers globally and not inside the function itself, so when we call a function multiple times it would not create new instances:

In [ ]:
# Convolutional modules
filters = 96

conv01 = layers.Conv1D(filters, 11, padding='same', activation="relu")
mp1 = layers.MaxPooling1D(3)
conv02 = layers.Conv1D(filters*2, 7, padding='same', activation="relu")
mp2 = layers.MaxPooling1D(3)
conv03 = layers.Conv1D(filters*4, 3, padding='same', activation="relu")
mp3 = layers.MaxPooling1D(3)
conv04 = layers.Conv1D(filters*2, 3, padding='same', activation="relu")
mp4 = layers.MaxPooling1D(3)

gru = layers.Bidirectional(layers.CuDNNGRU(filters, return_sequences=False))
if not gpus:
    gru = layers.Bidirectional(layers.GRU(filters, return_sequences=False))

def siamese_propagation(x):
    #Enter your code here to create a Siamese module
    
    return x

## 3. Merging point

The information from two sequences has to be merged at some point to produce a coherent prediction that would depend on the information from both proteins. There are many different ways to do that, but here we will consider an element-wise multiplication because a Biderectional GRU layer will produce only a single value for any given filter.

## 4. Prediction module

Here we have to create a *forward* function that takes two protein embeddings as inputs, processes them with a siamese module, merges two tensors using element-wise multiplication, and then uses two Dense layers to generate final output for binary classification.

*NB: Do not forget to use the sigmoid activation function in the last layer.*

In [ ]:
def forward(left, right):
    ## Enter your code here to create a forward propagation function
    ## Here we have to use the following functions/classes:
    ## siamese_propagation, layers.Dense, layers.multiply


## What metrics to use?
One of the problems of training a model for PPI prediction is the fact that the datasets are often unbalanced (there are much more negative interactions that positive ones). That is why, sometimes metrics like binary accuracy or ROC AUC might be not the best choice to evaluate the model's performance. Even though here we use a balanced dataset, we will still try to use more reliable metrics.

Here we will write three custom loss finctions for the accuracy, F1 score, and the Matthews correlation coefficient (MCC). These metrics provide more reliable assesments for imbalanced data.

$$
MCC = \frac{TP × TN - FP × FN}{\sqrt{(TP + FP)(TP + FN)(TN + FP)(TN + FN)}}
$$

$$
F_{1} = \frac{TP}{TP + \frac{1}{2}(FP + FN)}
$$

where $TP$ - true positives, $TN$ - true negatives, $FP$ - false positives, $FN$ - false negatives

The functions must take (y_true, y_pred) as two arguments, two tensors with shape (batch_size, 1) that represent the original labels and predictions respectively


In [ ]:
# For better compatibility one can use keras backend functions instead of numpy
# For example, the functions that you might need to use:
# K.sum instead of np.sum to sum up all the values in a tensor
# K.round instead of np.round to round every element to the closest integer in a tensor
# K.clip(x, 0, 1) to make sure that our values stay within [0, 1]
import keras.backend as K

##Enter you code here to create the 3 metrics: accuracy, f1, and mcc
    
def f1(y_true, y_pred):


def mcc(y_true, y_pred):


def accuracy(y_true, y_pred):

  

## The loss function

The classic loss function for binary classification problems is binary crossentropy. In this section we are going to write a function that will compute binary crossentropy having two tensors as an input (target values and predictions):

In [ ]:
def binary_crossentropy(y_true, y_pred):
    ## Enter your code here to create the loss fuction
    ## here we can again use keras.backend (K) module
    return loss

## The model initialization and training

After having prepared the architechture we can initialize the Model class and begin the learning on the preprocessed data:

In [ ]:
gc.collect()

model = Model(inputs=[seq_input1, seq_input2],
            outputs=[forward(seq_input1, seq_input2)])

adam = Adam(learning_rate=1e-4, amsgrad=True, epsilon=1e-6)

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint('/content/model.h5', monitor='val_mcc', mode='max')
earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)

model.compile(optimizer=adam, loss=binary_crossentropy, metrics=[accuracy, f1, mcc])

model.fit([x1_train, x2_train], y_train, epochs=50, callbacks=[checkpoint_callback, earlystop_callback],
          batch_size=64, verbose=1, validation_split=0.2)

# Data Analysis


## 1. Model testing
In this section we are going to get the predictions of the test data and compute the metrics we have used before. Moreover, we can build ROC and PR curves to estimate these metrics as well.

In [ ]:
model_saved = tf.keras.models.load_model('/content/model.h5', custom_objects={'f1': f1, 'mcc': mcc})

y_pred = model_saved.predict([x1_test, x2_test]).flatten()

In [ ]:
print('Test accuracy: ', accuracy(y_test, y_pred).numpy())
print('Test F1 score: ', f1(y_test,  y_pred).numpy())
print('Test MCC score: ', mcc(y_test, y_pred).numpy())

In [ ]:
from sklearn.metrics import PrecisionRecallDisplay

PrecisionRecallDisplay.from_predictions(y_test, y_pred)
plt.show()

In [ ]:
from sklearn.metrics import RocCurveDisplay

RocCurveDisplay.from_predictions(y_test, y_pred)
plt.show()

# C1 - C2 - C3 classes

The accuracy of PPI predictions greatly vary depending on the training and testing data. In some [state-of-the-art models](http://cb.csail.mit.edu/cb/dscript/) one can observe that the model performance becomes significantly worse when we test the model using proteins from organisms that are not related to the training set. Therefore, the performance of the model can be evaluated slightly differently: we can divide our testing set into 3 groups: C1 - pairs where both proteins were in the training data, C2 - pairs where only one of the proteins is in the training data, C3 - pairs that do not contain proteins from training data at all.

In this section you need to manually create three separate testing sets for C1-C2-C3 classes and check whether the model performance is any different.

In [ ]:
data = pd.read_csv('iBio-Summer-School/Interactions.tsv', sep='\t')
dict_seq = pd.read_csv('iBio-Summer-School/Dictionary.tsv', sep='\t')
dict_seq['Seq_string'] = dict_seq['Seq_string'].apply(blosum_emb)

#This function can be used to mimic the preprocessing step above
def preprocess_dataset(data):
  data = data.merge(dict_seq, left_on='Seq1', right_on='Id', how='left')
  data = data.merge(dict_seq, left_on='Seq2', right_on='Id', how='left', suffixes=('1', '2'))

  data = data[['Seq_string1', 'Seq_string2', 'Label']]

  x1 = np.asarray(data['Seq_string1'].tolist(), dtype=np.float16)
  x2 = np.asarray(data['Seq_string2'].tolist(), dtype=np.float16)
  y = np.asarray(data['Label'].tolist(), dtype=np.float32)

  return x1, x2, y

## Enter your code here to create a new training set (x1_train, x2_train, y_train)
## and 3 new test sets (x1_C1, x2_C1, y_C1), (x1_C2, x2_C2, y_C2), (x1_C3, x2_C3, y_C3)



In [ ]:
gc.collect()

model = Model(inputs=[seq_input1, seq_input2],
            outputs=[forward(seq_input1, seq_input2)])

adam = Adam(learning_rate=1e-4, amsgrad=True, epsilon=1e-6)

model.compile(optimizer=adam, loss='binary_crossentropy', metrics=[accuracy, f1, mcc])

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint('/content/model_C1_C2_C3.h5', monitor="val_loss")
earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)

model.fit([x1_train, x2_train], y_train, epochs=50, callbacks=[checkpoint_callback, earlystop_callback],
          batch_size=64, verbose=2, validation_split=0.2)

In [ ]:
model_saved = tf.keras.models.load_model('/content/model_C1_C2_C3.h5', custom_objects={'f1': f1, 'mcc': mcc})

y_pred_C1 = model_saved.predict([x1_C1, x2_C1]).flatten()
y_pred_C2 = model_saved.predict([x1_C2, x2_C2]).flatten()
y_pred_C3 = model_saved.predict([x1_C3, x2_C3]).flatten()

In [ ]:
## Enter your code here to test the model on three different test sets using previously defined metrics (e.g., accuracy)